## process pickle result for statistics info

In [68]:
import os
import pickle
import re
import numpy as np
# load pickle
class my_pickle:
    def __init__(self, file_path) -> None:
        with open(file_path, 'rb') as f:
            result = pickle.load(f)
        if isinstance(result, list):
            version = "old"
        elif isinstance(result, dict):
            version = "new"
        else:
            raise ValueError("not dict or list type")
        if version == "old":
            [self.correct_num, self.error_num, self.answers_list, self.contents_list] = result
        else:
            self.correct_num = result["correct_num"]
            self.error_num = result["error_num"]
            self.answers_list = result["answers_list"]
            self.contents_list = result["contents_list"]
        file_name = os.path.basename(file_path)
    
        
    def avg_acc(self, case_num, reason_num):
        correct_num = 0
        error_num = 0
        for case_id in range(case_num):
            case_answers = self.answers_list[case_id]
            for answer_id in range(reason_num):
                answer = case_answers[answer_id]
                if isinstance(answer, list):
                    if answer[1] == 1:
                        correct_num += 1  
                    else: 
                        error_num += 1
        print("acc:{}, correct num: {}, err num: {}".format(correct_num / (case_num * reason_num), correct_num, error_num))
        
        return correct_num / (case_num * reason_num), correct_num, error_num
    
    def std(self, case_num, reason_num):
        correct_num = [0 for _ in range(reason_num)]
        for case_id in range(case_num):
            case_answers = self.answers_list[case_id]
            case_answers = self.answers_list[case_id]
            for answer_id in range(reason_num):
                answer = case_answers[answer_id]
                if isinstance(answer, list):
                    if answer[1] == 1:
                        correct_num[answer_id] += 1  
        accs = [correct/case_num * 100 for correct in correct_num]
        print(accs)
        std = np.std(accs, ddof=1)
        mean = np.mean(accs)
        print("mean:{}, std:{}".format(mean, std))
                    
                    
    def vote(self, case_num, reason_num):
        from collections import Counter
        valid_count = 0
        vote_right_count = 0
        for case_id in range(case_num):
            answers = self.answers_list[case_id][:reason_num]
            answers = [answer for answer in answers if isinstance(answer, list)]  # clean answers
            if len(answers) == 0:
                continue
            else:
                valid_count += 1
            second_elements_are_1 = [sublist[1] == 1 for sublist in answers]
            any_second_element_is_1 = any(second_elements_are_1)
            if not any_second_element_is_1:
                continue
            true_answer = next((sublist[0] for sublist in answers if sublist[1] == 1), None)
            counter = Counter(sublist[0] for sublist in answers)
            guess_value, _ = counter.most_common(1)[0]
            if guess_value == true_answer:
                vote_right_count += 1

        print("vote_correct_num:{}, vaild_num:{}, vote_correct_num:{}".format(vote_right_count, valid_count, vote_right_count / valid_count))        
        

In [63]:
class my_log:
    def __init__(self, file_path) -> None:
        pattern = r'^index \d+/299, correct_num'
        self.error_num = 0
        self.correct_num = 0
        self.answers_list = []
        self.answers_content = []
        with open(file_path, 'r') as f:
            this_answer = []
            this_content = []
            content = ""
            plan_flag = 0
            for line in f:
                if line.startswith("right"):
                    this_answer.append(1)
                    self.error_num += 1
                if line.startswith("wrong"):
                    this_answer.append(0)
                    self.correct_num += 1
                if line.startswith("Plan end."):
                    plan_flag = 0
                    this_content.append(content)
                    content = ""
                if plan_flag:
                    content += line
                if line.startswith("Plan:"):
                    plan_flag = 1
                if bool(re.match(pattern, line)):
                    
                    self.answers_list.append(this_answer)
                    this_answer = []
                    self.answers_content.append(this_content)
                    this_content = []
                
    def acc(self, case_num):
        correct_num = 0 
        error_num = 0
        for i in range(case_num):
            answers = self.answers_list[i]
            for answer in answers:
                if answer == 1:
                    correct_num+=1
                else:
                    error_num+=1
        
        print("correct_num {}, error_num {}, acc {}".format(correct_num, error_num, correct_num/ (correct_num+error_num)))
    
    def vote(self, case_num):
        correct_case_num = 0 
        for i in range(case_num):
            correct_num = 0
            answers = self.answers_list[i]
            for answer in answers:
                if answer == 1:
                    correct_num+=1
            if correct_num >= 3:
                correct_case_num += 1
        print("correct_num {}, error_num {}, acc {}".format(correct_case_num, case_num - correct_case_num, correct_case_num/case_num))
        return 
        
        
            

In [75]:
file_path = ""
pck = my_pickle(file_path)